In [1]:
from peak_resolver.linear_combination_fitting import *
from peak_resolver.get_models_bases import get_models, get_basis_functions
from peak_resolver.process_chromeleon import process_chromeleon_file
import pickle
import pathlib
from pathlib import Path
import xarray as xr
from xarray import apply_ufunc

In [94]:
def set_bounds(composition):
    bounds = []
    for i in range(len(composition)):
        if composition[i] < 10**-5:
            # bounds.append([0,10**-5])
            bounds.append([0,0])
        else:
            bounds.append([0,2])
        
        

    return bounds



In [4]:
basis_root = '/Users/ianbillinge/Documents/yiplab/projects/org_acids_iems/Fanny Raw IC Data/CALIBRATION CURVES/calib.nc'
basis = get_basis_functions(basis_root, conc=0.5)
print([b.coords for b in basis])
basis = [i.to_numpy() for i in basis]




 Acetic dataset:



 Propanoic dataset:



 Butyric dataset:



 Valeric dataset:



 Caproic dataset:
[Coordinates:
  * t          (t) float64 6.501 6.504 6.507 6.511 ... 8.988 8.991 8.995 8.998
    Acetic     float64 0.5
    Propanoic  float64 0.0
    Butyric    float64 0.0
    Valeric    float64 0.0
    Caproic    float64 0.0, Coordinates:
  * t          (t) float64 6.501 6.504 6.507 6.511 ... 8.988 8.991 8.995 8.998
    Acetic     float64 0.0
    Propanoic  float64 0.5
    Butyric    float64 0.0
    Valeric    float64 0.0
    Caproic    float64 0.0, Coordinates:
  * t          (t) float64 6.501 6.504 6.507 6.511 ... 8.988 8.991 8.995 8.998
    Acetic     float64 0.0
    Propanoic  float64 0.0
    Butyric    float64 0.5
    Valeric    float64 0.0
    Caproic    float64 0.0, Coordinates:
  * t          (t) float64 6.501 6.504 6.507 6.511 ... 8.988 8.991 8.995 8.998
    Acetic     float64 0.0
    Propanoic  float64 0.0
    Butyric    float64 0.0
    Valeric    float64 0.5
    Capro

In [108]:
ds = xr.load_dataset('/Users/ianbillinge/Documents/yiplab/projects/org_acids_iems/Fanny Raw IC Data/2023-12-07.nc')
# ds['fitting_resid'] = xr.full_like(ds['s'],np.nan)
# ds['coeffs'] = xr.full_like(ds['s'],np.nan)

ds = ds.assign(resid=xr.zeros_like(ds["ds"]))
ds = ds.assign(p_ace=xr.zeros_like(ds["ds"]))
ds = ds.assign(p_but=xr.zeros_like(ds["ds"]))
ds = ds.assign(p_pro=xr.zeros_like(ds["ds"]))
ds = ds.assign(p_val=xr.zeros_like(ds["ds"]))
ds = ds.assign(p_cap=xr.zeros_like(ds["ds"]))
# p_ace = xr.DataArray(coords=ds['ds'].coords)
# p_but = xr.DataArray(coords=ds['ds'].coords)
# p_pro = xr.DataArray(coords=ds['ds'].coords)
# p_val = xr.DataArray(coords=ds['ds'].coords)
# p_cap = xr.DataArray(coords=ds['ds'].coords)


ds = ds.where(ds.t > 6.5).where(ds.t < 9)
ds = ds.dropna(dim='t', how='all')


In [109]:
ds.sel(side='F', exp_time=1,run=1,Acetic=50, Propionic=0, Butyric=0, Valeric=0, Caproic=0).dropna(dim='t',how='all')

<xarray.Dataset>
Dimensions:    (t: 761)
Coordinates:
  * t          (t) float64 6.501 6.504 6.507 6.511 ... 8.988 8.991 8.995 8.998
    exp_time   float64 1.0
    run        int32 1
    side       <U1 'F'
    Acetic     int32 50
    Propionic  int32 0
    Butyric    int32 0
    Valeric    int32 0
    Caproic    int32 0
Data variables:
    index      (t) float64 1.935e+03 1.936e+03 1.937e+03 ... 2.694e+03 2.695e+03
    s          (t) float64 -0.05655 -0.05634 -0.05612 ... 0.003178 0.002974
    ds         (t) float64 0.0001923 0.00022 0.0002195 ... -0.0001065 -0.0002078
    d2s        (t) float64 2.253e-06 2.817e-05 ... 2.221e-05 -0.0001033
    resid      (t) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0 0.0
    p_ace      (t) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0 0.0
    p_but      (t) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0 0.0
    p_pro      (t) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0 0.0
    p_val      (t) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0 0.0
    p_cap      (t) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0 0.0

In [134]:
for side in ds.coords['side'].values:
    for run in ds.coords['run'].values:
        for exp_time in ds.coords['exp_time'].values:
            for ace in ds.coords['Acetic'].values:
                for pro in ds.coords['Propionic'].values:
                    for but in ds.coords['Butyric'].values:
                        for val in ds.coords['Valeric'].values:
                            for cap in ds.coords['Caproic'].values:
                                current_selection = dict(
                                    side=side,
                                    run=run,
                                    exp_time=exp_time,
                                    Acetic=ace,
                                    Propionic=pro,
                                    Butyric=but,
                                    Valeric=val,
                                    Caproic=cap
                                )
                                
                                to_fit = ds.sel(current_selection).dropna(dim='t', how='all')['ds']
                                
                                try:
                                    composition = (ace, but, pro, val, cap)
                                    bbounds = set_bounds(composition)
                               
                                    res = lin_combination_fitting(to_fit.values, basis, bounds = bbounds)
                                    print('\ncomposition: {}'.format(composition))
                                    # print('bounds {}:'.format(bbounds)) 
                                    print('coeffs {}:'.format(res))
                                    # p_ace.loc[current_selection] = res['p'][0]
                                    # p_but.loc[current_selection] = res['p'][1]
                                    # p_pro.loc[current_selection] = res['p'][2]
                                    # p_val.loc[current_selection] = res['p'][3]
                                    # p_cap.loc[current_selection] = res['p'][4]
                                    if res['r'] > 0:
                                        ds['p_ace'].loc[current_selection] = res['p'][0]
                                        ds['p_but'].loc[current_selection] = res['p'][1]
                                        ds['p_pro'].loc[current_selection] = res['p'][2]
                                        ds['p_val'].loc[current_selection] = res['p'][3]
                                        ds['p_cap'].loc[current_selection] = res['p'][4]
                                    else:
                                        ds['p_ace'].loc[current_selection] = np.nan
                                        ds['p_but'].loc[current_selection] = np.nan
                                        ds['p_pro'].loc[current_selection] = np.nan
                                        ds['p_val'].loc[current_selection] = np.nan
                                        ds['p_cap'].loc[current_selection] = np.nan
                                        
                                except ValueError as ve:
                                    pass
                



/Users/ianbillinge/dev/peak_resolver/peak_resolver/linear_combination_fitting.py:58: OptimizeWarning: Initial guess is not within the specified bounds
  result = minimize(residuals, initial_guess, args=(target, vectors), bounds=bounds, method=method)
/Users/ianbillinge/dev/peak_resolver/peak_resolver/linear_combination_fitting.py:58: OptimizeWarning: Initial guess is not within the specified bounds
  result = minimize(residuals, initial_guess, args=(target, vectors), bounds=bounds, method=method)
/Users/ianbillinge/dev/peak_resolver/peak_resolver/linear_combination_fitting.py:58: OptimizeWarning: Initial guess is not within the specified bounds
  result = minimize(residuals, initial_guess, args=(target, vectors), bounds=bounds, method=method)
/Users/ianbillinge/dev/peak_resolver/peak_resolver/linear_combination_fitting.py:58: OptimizeWarning: Initial guess is not within the specified bounds
  result = minimize(residuals, initial_guess, args=(target, vectors), bounds=bounds, method=meth


composition: (0, 0, 0, 0, 0)
coeffs {'p': array([0., 0., 0., 0., 0.]), 'r': nan}:

composition: (0, 0, 0, 0, 50)
coeffs {'p': array([0. , 0. , 0. , 0. , 0.5]), 'r': nan}:

composition: (0, 0, 0, 50, 0)
coeffs {'p': array([0.        , 0.        , 0.        , 0.31780313, 0.        ]), 'r': 0.014481951283494818}:

composition: (0, 0, 0, 50, 50)
coeffs {'p': array([0. , 0. , 0. , 0.5, 0.5]), 'r': nan}:

composition: (0, 50, 0, 0, 0)
coeffs {'p': array([0. , 0.5, 0. , 0. , 0. ]), 'r': nan}:

composition: (0, 50, 0, 0, 50)
coeffs {'p': array([0. , 0.5, 0. , 0. , 0.5]), 'r': nan}:


/Users/ianbillinge/dev/peak_resolver/peak_resolver/linear_combination_fitting.py:58: OptimizeWarning: Initial guess is not within the specified bounds
  result = minimize(residuals, initial_guess, args=(target, vectors), bounds=bounds, method=method)
/Users/ianbillinge/dev/peak_resolver/peak_resolver/linear_combination_fitting.py:58: OptimizeWarning: Initial guess is not within the specified bounds
  result = minimize(residuals, initial_guess, args=(target, vectors), bounds=bounds, method=method)
/Users/ianbillinge/dev/peak_resolver/peak_resolver/linear_combination_fitting.py:58: OptimizeWarning: Initial guess is not within the specified bounds
  result = minimize(residuals, initial_guess, args=(target, vectors), bounds=bounds, method=method)
/Users/ianbillinge/dev/peak_resolver/peak_resolver/linear_combination_fitting.py:58: OptimizeWarning: Initial guess is not within the specified bounds
  result = minimize(residuals, initial_guess, args=(target, vectors), bounds=bounds, method=meth


composition: (0, 50, 0, 50, 0)
coeffs {'p': array([0. , 0.5, 0. , 0.5, 0. ]), 'r': nan}:

composition: (0, 50, 0, 50, 50)
coeffs {'p': array([0. , 0.5, 0. , 0.5, 0.5]), 'r': nan}:

composition: (0, 0, 50, 0, 0)
coeffs {'p': array([0. , 0. , 0.5, 0. , 0. ]), 'r': nan}:

composition: (0, 0, 50, 0, 50)
coeffs {'p': array([0. , 0. , 0.5, 0. , 0.5]), 'r': nan}:

composition: (0, 0, 50, 50, 0)
coeffs {'p': array([0. , 0. , 0.5, 0.5, 0. ]), 'r': nan}:


/Users/ianbillinge/dev/peak_resolver/peak_resolver/linear_combination_fitting.py:58: OptimizeWarning: Initial guess is not within the specified bounds
  result = minimize(residuals, initial_guess, args=(target, vectors), bounds=bounds, method=method)
/Users/ianbillinge/dev/peak_resolver/peak_resolver/linear_combination_fitting.py:58: OptimizeWarning: Initial guess is not within the specified bounds
  result = minimize(residuals, initial_guess, args=(target, vectors), bounds=bounds, method=method)
/Users/ianbillinge/dev/peak_resolver/peak_resolver/linear_combination_fitting.py:58: OptimizeWarning: Initial guess is not within the specified bounds
  result = minimize(residuals, initial_guess, args=(target, vectors), bounds=bounds, method=method)
/Users/ianbillinge/dev/peak_resolver/peak_resolver/linear_combination_fitting.py:58: OptimizeWarning: Initial guess is not within the specified bounds
  result = minimize(residuals, initial_guess, args=(target, vectors), bounds=bounds, method=meth

...


composition: (50, 0, 0, 50, 50)
coeffs {'p': array([0.5, 0. , 0. , 0.5, 0.5]), 'r': nan}:

composition: (50, 50, 0, 0, 0)
coeffs {'p': array([0.5, 0.5, 0. , 0. , 0. ]), 'r': nan}:

composition: (50, 50, 0, 0, 50)
coeffs {'p': array([0.5, 0.5, 0. , 0. , 0.5]), 'r': nan}:

composition: (50, 50, 0, 50, 0)
coeffs {'p': array([0.5, 0.5, 0. , 0.5, 0. ]), 'r': nan}:

composition: (50, 50, 0, 50, 50)
coeffs {'p': array([0.5, 0.5, 0. , 0.5, 0.5]), 'r': nan}:


/Users/ianbillinge/dev/peak_resolver/peak_resolver/linear_combination_fitting.py:58: OptimizeWarning: Initial guess is not within the specified bounds
  result = minimize(residuals, initial_guess, args=(target, vectors), bounds=bounds, method=method)
/Users/ianbillinge/dev/peak_resolver/peak_resolver/linear_combination_fitting.py:58: OptimizeWarning: Initial guess is not within the specified bounds
  result = minimize(residuals, initial_guess, args=(target, vectors), bounds=bounds, method=method)
/Users/ianbillinge/dev/peak_resolver/peak_resolver/linear_combination_fitting.py:58: OptimizeWarning: Initial guess is not within the specified bounds
  result = minimize(residuals, initial_guess, args=(target, vectors), bounds=bounds, method=method)
/Users/ianbillinge/dev/peak_resolver/peak_resolver/linear_combination_fitting.py:58: OptimizeWarning: Initial guess is not within the specified bounds
  result = minimize(residuals, initial_guess, args=(target, vectors), bounds=bounds, method=meth


composition: (50, 0, 50, 0, 0)
coeffs {'p': array([0.5, 0. , 0.5, 0. , 0. ]), 'r': nan}:

composition: (50, 0, 50, 0, 50)
coeffs {'p': array([0.5, 0. , 0.5, 0. , 0.5]), 'r': nan}:

composition: (50, 0, 50, 50, 0)
coeffs {'p': array([0.5, 0. , 0.5, 0.5, 0. ]), 'r': nan}:

composition: (50, 0, 50, 50, 50)
coeffs {'p': array([0.5, 0. , 0.5, 0.5, 0.5]), 'r': nan}:

composition: (50, 50, 50, 0, 0)
coeffs {'p': array([0.5, 0.5, 0.5, 0. , 0. ]), 'r': nan}:


/Users/ianbillinge/dev/peak_resolver/peak_resolver/linear_combination_fitting.py:58: OptimizeWarning: Initial guess is not within the specified bounds
  result = minimize(residuals, initial_guess, args=(target, vectors), bounds=bounds, method=method)
/Users/ianbillinge/dev/peak_resolver/peak_resolver/linear_combination_fitting.py:58: OptimizeWarning: Initial guess is not within the specified bounds
  result = minimize(residuals, initial_guess, args=(target, vectors), bounds=bounds, method=method)
/Users/ianbillinge/dev/peak_resolver/peak_resolver/linear_combination_fitting.py:58: OptimizeWarning: Initial guess is not within the specified bounds
  result = minimize(residuals, initial_guess, args=(target, vectors), bounds=bounds, method=method)
/Users/ianbillinge/dev/peak_resolver/peak_resolver/linear_combination_fitting.py:58: OptimizeWarning: Initial guess is not within the specified bounds
  result = minimize(residuals, initial_guess, args=(target, vectors), bounds=bounds, method=meth


composition: (50, 50, 50, 0, 50)
coeffs {'p': array([0.5, 0.5, 0.5, 0. , 0.5]), 'r': nan}:

composition: (50, 50, 50, 50, 0)
coeffs {'p': array([0.5, 0.5, 0.5, 0.5, 0. ]), 'r': nan}:

composition: (50, 50, 50, 50, 50)
coeffs {'p': array([0.5, 0.5, 0.5, 0.5, 0.5]), 'r': nan}:


In [138]:
ds['p_ace'].sel(side='F', exp_time=1, run=1, Acetic=50, Propionic=0, Butyric=0, Valeric=0,Caproic=0)

<xarray.DataArray 'p_ace' (t: 761)>
array([0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
...
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377, 0.46313377, 0.46313377, 0.46313377, 0.46313377,
       0.46313377])
Coordinates:
  * t          (t) float64 6.501 6.504 6.507 6.511 ... 8.988 8.991 8.995 8.998
    exp_time   float64 1.0
    run        int32 1
    side       <U1 'F'
    Acetic     int32 50
    Propionic  int32 0
    Butyric    int32 0
    Valeric    int32 0
    Caproic    int32 0

In [142]:
df = ds.to_dataframe()
df = df.where(df['s'] > 0).dropna()
df.to_csv('/Users/ianbillinge/Desktop/massive.csv')

ModuleNotFoundError: No module named 'openpyxl'

In [ ]:
def vectorized_lin_combination_fitting(
    target,
    vectors, 
    dim='t', 
    initial_guess=None, 
    bounds = None, 
    method='Nelder-Mead'
):

    return apply_ufunc(lin_combination_fitting, target, vectors, input_core_dims=[[dim], [dim]])


In [ ]:
targ = ds.sel(exp_time=1, run=1, side='F', Acetic=50, Butyric=0, Propionic=0, Valeric=0, Caproic=0)
targ=targ.where(targ.t > 6.5).where(targ.t < 9)
targ = targ.dropna(dim='t', how='all')['ds']
targ = targ.values


In [ ]:
ds

In [ ]:
res = vectorized_lin_combination_fitting(ds, basis[0:5], initial_guess=(1,0.5, 0.5, 0.5, 0.5))
res

In [ ]:
def mean(data, dim="t"):

    return apply_ufunc(

        np.abs,

        data,

        input_core_dims=[dim],

        vectorize=True,

    )


    

In [ ]:
mean = mean(ds['s'])

In [ ]:
mean

In [ ]:
df = ds['ds'].to_dataframe().dropna()

In [ ]:
lin_combination_fitting?

In [ ]:
root = '/Users/ianbillinge/Documents/yiplab/projects/org_acids_iems/Fanny Raw IC Data/Calibration Curves'
acids = ['acetic', 'propanoic', 'butyric', 'valeric', 'caproic']
models = get_models(root, acids)
acids = [a.capitalize() for a in acids]
for i in list(models):
    models[i + '_err'] = np.sqrt(np.sum(models[i].resid)**2/models[i].df_resid)

In [ ]:
models['ace'].resid

In [ ]:
models

In [ ]:
a = xr.DataArray([[1,2,3], [4,5,6]], dims=['x', 'y'])
b = xr.DataArray([[-1, -2, -3], [4,5,6]], dims=['x', 'y'])

In [ ]:
b

In [ ]:
def magnitude(a, b):

    # func = lambda x, y: np.sqrt(x**2 + y**2)

    return xr.apply_ufunc(norm, a, b)

def norm(x, y):
    return np.sqrt(x**2 + y**2)

def _inner(x, y):

    result = np.matmul(x[..., np.newaxis, :], y[..., :, np.newaxis])

    return result[..., 0, 0]


def inner_product(a, b, dim):

    return apply_ufunc(_inner, a, b, input_core_dims=[[dim], [dim]])



In [ ]:
v_lin_com()

In [ ]:
a

In [ ]:
b

In [ ]:
inner_product(a,b,'y')